# Simulation Study: Closed System (Bootstrap)

In [1]:
### LIBRARY ###
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random as rand

from mm_temptations_functions import *
from scipy.optimize import minimize, Bounds, LinearConstraint

from multiprocess import Pool
from functools import partial

import pickle

In [2]:
planting(456)

In [3]:
# set system wide param
k = 7 # no. of states
timeCourse = 16 # total timepoints
u = [1] * int(timeCourse) # time interval
n_param = int(k/2 * (k-1)) # number of parameters

sampRange_ub = 4
xterm = 1e-15
max_iter = 5e3
numCores = 5
mc_iter = 5

In [4]:
file = "simResults/simulated_CN_Q_true.pkl"
with open(file, 'rb') as f:
    Q_true = pickle.load(f)

In [5]:
filename = "simDatasets/simulated_CN_propDicts_groupSize5.pkl"
with open(filename, 'rb') as f:
    [pi_hat_donors, T_donors, u_donors] = pickle.load(f)

In [6]:
filename = "simResults/simulated_CN_estimates_50reps.pkl"
with open(filename, 'rb') as f:
                [estimated_theta, estimated_cost] = pickle.load(f)

In [7]:
point_estimate = estimated_theta[0,:]

### Bootstrap Uncertainty

In [8]:
# set variables
Q_template = make_theta0(k, 'Q')

reg_bounds = def_bounds(n_param, k, account_ingress=False)
reg_con = def_constraints(n_param, k)

In [9]:
B = 1500
startIteration = 985
optimiserArgs = {'costFunc': calc_cost_donors, 'args': (pi_hat_donors, T_donors, k, u_donors, Q_template), 'bounds': reg_bounds, 'constraints': reg_con, 'sampRange_ub': sampRange_ub}
options = {'hessian': None, 'xterm': xterm, 'max_iter': max_iter}

mcArgs = {'mciter': mc_iter, 'n_cores': numCores}

In [10]:
# create new checkpoint array
#theta_boot = np.zeros((B,n_param))
#checkpoint_filename = "simResults/simCN_BScheckpoint_theta_boot.pkl"   
#with open(checkpoint_filename, 'wb') as f:
#    pickle.dump(theta_boot,f)

In [11]:
# load old check point into new checkpoint
theta_boot = np.zeros((B,n_param))
checkpoint_filename = "simResults/simCN_BScheckpoint_theta_boot.pkl"
with open(checkpoint_filename, 'rb') as f:
    theta_boot = pickle.load(f)
    
with open(checkpoint_filename, 'wb') as f:
    pickle.dump(theta_boot,f)

In [12]:
residResamp=bootstrap_resResamp_perDonor(point_estimate=point_estimate, n_bootstrapSamples=B, n_param=n_param, theta_generator=make_theta0, mcArgs=mcArgs, optimiserArgs=optimiserArgs, options=options, checkpoint_filename=checkpoint_filename, startIteration=startIteration)

 42% (220 of 515) |##    | Elapsed Time: 5 days, 4:01:23 ETA:  8 days, 19:52:12
 42% (221 of 515) |###    | Elapsed Time: 5 days, 4:31:47 ETA:  6 days, 4:55:35
 43% (222 of 515) |###    | Elapsed Time: 5 days, 4:42:41 ETA:  2 days, 5:18:07
 43% (223 of 515) |##    | Elapsed Time: 5 days, 5:30:30 ETA:  9 days, 16:40:25
 43% (224 of 515) |##    | Elapsed Time: 5 days, 5:55:02 ETA:  4 days, 22:57:12
 43% (225 of 515) |###    | Elapsed Time: 5 days, 6:25:04 ETA:  6 days, 1:13:22
 43% (226 of 515) |##   | Elapsed Time: 5 days, 7:22:44 ETA:  11 days, 13:41:46
 44% (227 of 515) |###    | Elapsed Time: 5 days, 8:04:32 ETA:  8 days, 8:41:39
 44% (228 of 515) |##    | Elapsed Time: 5 days, 8:23:17 ETA:  3 days, 17:41:06
 44% (229 of 515) |###    | Elapsed Time: 5 days, 8:59:11 ETA:  7 days, 3:08:24
 44% (230 of 515) |##    | Elapsed Time: 5 days, 9:14:20 ETA:  2 days, 23:54:55
 44% (231 of 515) |#   | Elapsed Time: 5 days, 10:14:10 ETA:  11 days, 19:15:14
 45% (232 of 515) |#   | Elapsed Time: 5

Compiling results...


In [13]:
filename = "simResults/simCN_BS_allTime_allDonor_5state.pkl"
with open(filename, 'wb') as f:
    pickle.dump(residResamp, f)